# Retrieval results

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from netCDF4 import Dataset
import glob
import matplotlib.pyplot as plt
import numpy as np
from joint_flight.results import get_results, plot_results

In [ ]:
shapes = ["6-BulletRosette",
          "LargeColumnAggregate",
          "8-ColumnAggregate",
          "LargePlateAggregate",
          "EvansSnowAggregate"]
results_b984_ro = get_results("b984", "radar_only", "Radar only")
results_b984 = get_results("b984")
results_c159_ro = get_results("c159", "radar_only", "Radar only")
results_c159 = get_results("c159")

## Residual distributions

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

from joint_flight.data.b984 import RADAR as radar_b984
from joint_flight.data.c159 import RADAR as radar_c159
from joint_flight.data.c161 import RADAR as radar_c161
from joint_flight.data.b984 import ATMOSPHERE as atmosphere_b984
from joint_flight.data.c159 import ATMOSPHERE as atmosphere_c159
from joint_flight.data.c161 import ATMOSPHERE as atmosphere_c161

m = len(shapes) + 1
gs = GridSpec(m, 5,
              width_ratios=[0.1, 1.0, 1.0, 1.0, 0.1],
              height_ratios=[1.5] + len(shapes) * [0.75])
f = plt.figure(figsize=(15, 12))
names = [f.add_subplot(gs[i, 0]) for i in range(m)]
axs_b984 = [f.add_subplot(gs[i, 1]) for i in range(m)]
axs_c159 = [f.add_subplot(gs[i, 2]) for i in range(m)]
axs_c161 = [f.add_subplot(gs[i, 3]) for i in range(m)]
legends = [f.add_subplot(gs[:2, -1]), f.add_subplot(gs[2:, -1])]

plot_results(radar_b984, results_b984, atmosphere_b984, axs=axs_b984, legends=legends, shapes=shapes, names=names)
plot_results(radar_b984, results_b984_ro, atmosphere_b984, axs=axs_c159, shapes=shapes, y_axis=False)
axs_b984[0].set_title("B984", weight="bold", pad=15)
axs_c159[0].set_title("B984 (RO)", weight="bold", pad=15)

plt.tight_layout()
plt.show()
f.savefig(
    "../plots/ice_water_content_ro.png",
    bbox_inches="tight"
)

## Comparison to nevzorov data

In [ ]:
from joint_flight.data.b984 import NEVZOROV as nevzorov_b984
from joint_flight.data.c159 import NEVZOROV as nevzorov_c159
from joint_flight.results import match_bulk_properties

In [ ]:
from joint_flight.results import get_distance_mask, get_domain_mask
results_b984 = {k: results_b984[k] for k in shapes}
results_b984_ro = {k: results_b984_ro[k] for k in shapes}
mask_b984 = get_distance_mask(radar_b984, nevzorov_b984)
matched_b984 = match_bulk_properties(results_b984,
                                     mask_b984,
                                     radar_b984,
                                     nevzorov_b984)
matched_b984_ro = match_bulk_properties(results_b984_ro,
                                     mask_b984,
                                     radar_b984,
                                     nevzorov_b984)

results_c159 = {k: results_c159[k] for k in shapes}
mask_c159 = get_domain_mask(radar_c159, 50e3, 200e3, nevzorov_c159)
matched_c159 = match_bulk_properties(results_c159,
                                     mask_c159,
                                     radar_c159,
                                     nevzorov_c159)
matched_c159_ro = match_bulk_properties(results_c159_ro,
                                        mask_c159,
                                        radar_c159,
                                        nevzorov_c159)

In [ ]:
from joint_flight.results import plot_bulk_properties
from matplotlib.gridspec import GridSpec

f = plt.figure(figsize=(15, 9))
gs = GridSpec(2, 5, height_ratios=[0.4, 1.0], width_ratios=[0.1, 1.0, 1.0, 0.05, 1.5], wspace=0.2, hspace=0.3)
axs_b984 = [f.add_subplot(gs[i, 1]) for i in range(2)]
axs_b984_ro = [f.add_subplot(gs[i, 2]) for i in range(2)]
cbs = [f.add_subplot(gs[i, 3]) for i in range(2)]
legends = [f.add_subplot(gs[i, 4]) for i in range(2)]

plot_bulk_properties(nevzorov_b984,
                     matched_b984,
                     radar_b984,
                     mask_b984,
                     cbs=cbs,
                     shapes=shapes,
                     axs=axs_b984)
plot_bulk_properties(nevzorov_b984,
                     matched_b984_ro,
                     radar_b984,
                     mask_b984,
                     axs=axs_b984_ro,
                     cbs=cbs,
                     shapes=shapes,
                     legends=legends,
                     y_axis=False)
axs_b984[0].remove()
axs_b984_ro[0].remove()
cbs[0].remove()
axs_b984[1].set_title("(a) Combined", loc="left", pad=15)
axs_b984_ro[1].set_title("(b) Radar only", loc="left", pad=15)
legends[0].remove()
f.savefig(
    "../plots/in_situ_iwc_radar_only.png",
    dpi=300
)

In [ ]:
from joint_flight.results import plot_bulk_properties
from matplotlib.gridspec import GridSpec

f = plt.figure(figsize=(15, 9))
gs = GridSpec(2, 5, height_ratios=[0.4, 1.0], width_ratios=[0.1, 1.0, 1.0, 0.05, 1.5], wspace=0.2, hspace=0.3)
axs_c159 = [f.add_subplot(gs[i, 1]) for i in range(2)]
axs_c159_ro = [f.add_subplot(gs[i, 2]) for i in range(2)]
cbs = [f.add_subplot(gs[i, 3]) for i in range(2)]
legends = [f.add_subplot(gs[i, 4]) for i in range(2)]


plot_bulk_properties(nevzorov_c159,
                     matched_c159,
                     radar_c159,
                     mask_c159,
                     cbs=cbs,
                     shapes=shapes,
                     axs=axs_c159)
plot_bulk_properties(nevzorov_c159,
                     matched_c159_ro,
                     radar_c159,
                     mask_c159,
                     axs=axs_c159_ro,
                     cbs=cbs,
                     shapes=shapes,
                     legends=legends,
                     y_axis=False)
axs_c159[0].remove()
axs_c159_ro[0].remove()
cbs[0].remove()
axs_c159[1].set_title("(a) Combined", loc="left", pad=15)
axs_c159_ro[1].set_title("(b) Radar only", loc="left", pad=15)
legends[0].remove()
f.savefig("../plots/in_situ_iwc_radar_only_c159.png", dpi=300)

In [ ]:
from joint_flight.results import calculate_psds
psd_r_b984 = calculate_psds(results_b984,
                            mask_b984,
                            radar_b984)
psd_r_b984_ro = calculate_psds(results_b984_ro,
                               mask_b984,
                               radar_b984)
psd_r_c159 = calculate_psds(results_c159,
                            mask_c159,
                            radar_c159)
psd_r_c159_ro = calculate_psds(results_c159_ro,
                               mask_c159,
                               radar_c159)

In [ ]:
from joint_flight.data.b984 import PSD as psd_b984
from joint_flight.results import plot_psds
psd_b984 = psd_b984.resample(time="10s").mean()
psd_b984_ro = psd_b984.resample(time="10s").mean()
nevzorov_b984 = nevzorov_b984.resample(time="10s").mean()
nevzorov_b984_ro = nevzorov_b984.resample(time="10s").mean()

fig = plt.figure(figsize=(12, 12))
gs = GridSpec(6, 4, width_ratios=[0.8, 1.0, 1.0, 0.5], wspace=0.1, hspace=0.1)


names_axs = [plt.subplot(gs[i, 0]) for i in range(6)]
legend_axs = [plt.subplot(gs[i, -1]) for i in range(6)]
axs_b984 = [plt.subplot(gs[i, 1]) for i in range(6)]
axs_b984_ro = [plt.subplot(gs[i, 2], sharey=axs_b984[i]) for i in range(6)]

plot_psds(psd_b984,
          psd_r_b984,
          axs=axs_b984,
          legends=legend_axs,
          shapes=shapes,
          names=names_axs)
plot_psds(psd_b984,
          psd_r_b984_ro,
          axs=axs_b984_ro,
          legends=legend_axs,
          names=names_axs,
          shapes=shapes,
          y_axis=False)
axs_b984[0].set_title("(a) Combined", loc="left", pad=15)
axs_b984_ro[0].set_title("(b) Radar only", loc="left", pad=15)
plt.show()
fig.savefig(
    "../plots/in_situ_psds_radar_only.png",
    bbox_inches="tight",
    dpi=300
)

In [ ]:
from joint_flight.data.c159 import PSD as psd_c159
from joint_flight.results import plot_psds
psd_c159 = psd_c159.resample(time="10s").mean()
psd_c159_ro = psd_c159.resample(time="10s").mean()
nevzorov_c159 = nevzorov_c159.resample(time="10s").mean()
nevzorov_c159_ro = nevzorov_c159.resample(time="10s").mean()

fig = plt.figure(figsize=(12, 12))
gs = GridSpec(6, 4, width_ratios=[0.8, 1.0, 1.0, 0.5], wspace=0.1, hspace=0.1)


names_axs = [plt.subplot(gs[i, 0]) for i in range(6)]
legend_axs = [plt.subplot(gs[i, -1]) for i in range(6)]
axs_c159 = [plt.subplot(gs[i, 1]) for i in range(6)]
axs_c159_ro = [plt.subplot(gs[i, 2], sharey=axs_c159[i]) for i in range(6)]

plot_psds(psd_c159,
          psd_r_c159,
          axs=axs_c159,
          legends=legend_axs,
          shapes=shapes,
          names=names_axs)
plot_psds(psd_c159,
          psd_r_c159_ro,
          axs=axs_c159_ro,
          legends=legend_axs,
          names=names_axs,
          shapes=shapes,
          y_axis=False)
axs_c159[0].set_title("(a) Combined", loc="left", pad=15)
axs_c159_ro[0].set_title("(b) Radar only", loc="left", pad=15)
plt.show()
fig.savefig(
    "../plots/in_situ_psds_radar_only_c159.png",
    bbox_inches="tight",
    dpi=300
)

In [ ]:
from joint_flight.results import plot_psd_mass
from joint_flight.data.b984 import PSD as psd_b984
from joint_flight.data.c159 import PSD as psd_c159
from joint_flight.results import plot_psds
psd_b984 = psd_b984.resample(time="10s").mean()
psd_c159 = psd_c159.resample(time="10s").mean()
nevzorov_b984 = nevzorov_b984.resample(time="10s").mean()
nevzorov_c159 = nevzorov_c159.resample(time="10s").mean()

fig = plt.figure(figsize=(12, 7))
gs = GridSpec(1, 4, width_ratios=[1.2, 1.2, 0.075, 0.5])

axs_b984 = [fig.add_subplot(gs[i, 0]) for i in range(1)]
axs_c159 = [fig.add_subplot(gs[i, 1]) for i in range(1)]
cbs = [fig.add_subplot(gs[i, 2]) for i in range(1)]
legend_axs = [fig.add_subplot(gs[i, 3]) for i in range(1)]

plot_psd_mass(psd_b984,
              nevzorov_b984,
              axs=axs_b984,
              legends=legend_axs,
              shapes=shapes,
              cbs=cbs,
              names=names_axs)
plot_psd_mass(psd_c159,
              nevzorov_c159,
              axs=axs_c159,
              legends=legend_axs,
              shapes=shapes,
              cbs=cbs,
              names=names_axs,
              y_axis=False)
axs_b984[0].set_title("(a) B984", loc="left", pad=15)
axs_c159[0].set_title("(b) C159", loc="left", pad=15)
plt.tight_layout()
plt.show()

fig.savefig(
    "../plots/mass_size_relations.png",
    bbox_inches="tight",
    dpi=200
)